In [1]:
import requests
import json
import os
import tarfile
from urllib.parse import urlparse
from datetime import datetime, timedelta
import rasterio as rio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
from rasterio.transform import xy

In [2]:
# static input
env = {}
env['static_input'] = {'start_date':'2000-11-21', 'end_date':'2000-11-21', 'horizontal':'004','vertical':'002'}
env['predecessor_outputs'] = {}

In [3]:
context = {}

In [10]:
os.environ['USERNAME'] = ''
os.environ['PASSWORD'] = ''
os.environ['REGION'] = 'CU'
os.environ['GEOJSON'] = 'geojson/meadow.geojson'
os.environ['DATA_SET_NAME'] = 'ARD_TILE'
os.environ['PRODUCT'] = 'SR'
os.environ['FILEPATH'] = './tmp/'

In [15]:
class Phenology: 
    def get_apikey(username, password):
        payload = dict(username=username, password=password, catalogId='EE', authType='EROS')
        data = dict(jsonRequest=json.dumps(payload))
        r = requests.post('https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/login', data=data)
        creds = r.json()
        return creds['data']
    
    def get_scenes(apiKey, region, horizontal, vertical, start_date, end_date):
        myfilter = {
                "filterType": "and", 
                "childFilters": [
                    {"filterType":"value","fieldId":21789,"value":region}, #Grid Region
                    {"filterType":"value","fieldId":21787,"value":horizontal}, #Horizontal
                    {"filterType":"value","fieldId":21788,"value":vertical}, #Vertical
                ]
            }
        startDate, endDate = Phenology.get_dates(start_date, end_date)
        temporalfilter = {
            "startDate": startDate,
            "endDate": endDate
        }
        request_code = 'search'
        baseurl = f'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/{request_code}'
        payload = {'apiKey':apiKey, 
                   'datasetName': os.environ['DATA_SET_NAME'], 
                   "temporalFilter": temporalfilter,
                   'additionalCriteria': myfilter,  
                   'maxResults':10000}
        data = dict(jsonRequest=json.dumps(payload))
        r = requests.get(baseurl, params=data)
        response = r.json()
        return response
    
    # 15 day buffer before start date and after end date
    def get_dates(start_date, end_date):
        start_date = datetime(int(start_date[:4]), int(start_date[5:7]), int(start_date[-2:]))
        end_date = datetime(int(end_date[:4]), int(end_date[5:7]), int(end_date[-2:]))
        start = (start_date - timedelta(days=15)).strftime('%Y-%m-%d')
        end = (end_date + timedelta(days=15)).strftime('%Y-%m-%d')
        return start, end
    
    def get_url(entityId, apiKey, product=os.environ['PRODUCT']):
        request_code = 'download'
        baseurl = f'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/{request_code}'
        payload = {'apiKey':apiKey, 
                   'datasetName': os.environ['DATA_SET_NAME'], 
                   'entityIds':[entityId],
                   'products':[product]} #these are 'download codes' from above
        data = dict(jsonRequest=json.dumps(payload))
        r = requests.get(baseurl, params=data)
        data = r.json()['data']
        download_url = r.json()['data'][0]['url']
        p = urlparse(download_url)
        name = os.path.basename(p.path)
        return os.environ['FILEPATH'] + name, download_url
    
    def download(filename, url):
        chunks = 1024*1024
        with requests.get(url, stream=True) as r:
            with open(filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=chunks):
                    f.write(chunk)
    
    def calculate_NDVI(nir_data, red_data, longitudes, latitudes):
        np.seterr(invalid='ignore')
        x, y = np.meshgrid(longitudes, latitudes)
        nir_data = nir_data.astype(float)
        nir_data = nir_data.astype(float)
        data = np.array([nir_data.flatten(), red_data.flatten(), x.flatten(), y.flatten()])
        data = np.rot90(data)
        # filter out negative values and values greater than 10,000
        data = data[np.logical_and(np.logical_and(data[:, 0] >= 0, data[:, 1] >= 0), np.logical_and(data[:, 0] <= 10000, data[:, 1] <= 10000))]
        ndvi = np.array([np.divide(np.subtract(data[:, 0], data[:, 1]), np.add(data[:, 0], data[:, 1]))])
        ndvi = np.rot90(ndvi)
        data = np.append(data, ndvi, 1)
        # filter out invalid ndvi values
        data = data[np.logical_and(data[:, 4] >= -1, data[:, 4] <= 1)]
        # calculation of values
        maximum = data[:, 4].max() if len(data) != 0 else 'invalid'
        flowering = data[np.logical_and(data[:, 4] <= .425, data[:, 4] >= .375)]
        FF = round(len(flowering)/len(data), 2) if len(data) != 0 else 'invalid'
        VF = 1 - FF if len(data) != 0 else 'invalid'
        return {
            'avg': round(data[:, 4].mean(), 2) if len(data) != 0 else 'invalid',
            'max': round(maximum, 2) if len(data) != 0 else 'invalid',
            'min': round(data[:, 4].min(), 2) if len(data) != 0 else 'invalid',
            'std': round(data[:, 4].std(), 2) if len(data) != 0 else 'invalid',
            'FF': FF,
            'VF': VF,
            'coordinates': json.dumps(data[data[:, 4] == maximum][:, 2:4].round(4).tolist()) if len(data) != 0 else 'invalid'
        }
        
    def get_reflectance(data):
        data = data.astype(float)
        # filters values between 0 and 10000
        data = np.divide(data[np.logical_and(data >= 0,data <= 10000)], 10000)
        return {'avg': round(data.mean(), 2) if data.size != 0 else 'invalid', 
                'max': round(data.max(), 2) if data.size != 0 else 'invalid', 
                'min': round(data.min(), 2) if data.size != 0 else 'invalid', 
                'std': round(data.std(), 2) if data.size != 0 else 'invalid'
               }
    
    def get_statistical_values(tar):
        values = {}
        with tarfile.open(tar) as tar:
            for member in tar.getmembers():
                if member.name[-8:] in ['SRB5.tif','SRB4.tif', 'SRB3.tif', 'SRB2.tif']:
                    path = os.environ['FILEPATH'] + member.name
                    tar.extract(member, path=os.environ['FILEPATH'])
                    outpath = Phenology.reproject(path, path + 'cropped')
                    data = Phenology.crop_raster_data(outpath)
                    if member.name[-8:] == 'SRB4.tif':   
                        values['red_data'] = data
                        with rio.open(outpath) as src:
                            transform = src.meta['transform']
                            values['longitudes'] = xy(transform, [x for x in range(src.width)], [y for y in range(src.width)])[0]
                            values['latitudes'] = xy(transform, [x for x in range(src.height)], [y for y in range(src.height)])[1]
                    if member.name[-8:] == 'SRB3.tif':
                        values['green_data'] = data
                    if member.name[-8:] == 'SRB2.tif':
                        values['blue_data'] = data
                    if member.name[-8:] == 'SRB5.tif':
                        values['nir_data'] = data
        return {'blue_ref': Phenology.get_reflectance(values['blue_data']), 
                'green_ref': Phenology.get_reflectance(values['green_data']), 
                'red_ref': Phenology.get_reflectance(values['red_data']), 
                'NDVI': Phenology.calculate_NDVI(values['nir_data'], 
                                                 values['red_data'], 
                                                 values['longitudes'], 
                                                 values['latitudes'])}

    def reproject(inpath, outpath, dst_crs='EPSG:4326'):
        with rio.open(inpath) as src:
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })

            with rio.open(outpath, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rio.band(src, i),
                        destination=rio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)
        return outpath
                    
    # returns cropped data and crops the tif file
    def crop_raster_data(tif_path):
        with rio.open(tif_path) as src:
            extent_geojson = env['predecessor_outputs']['2']['extent_geojson']
            out_image, out_transform = mask(src, shapes=[json.loads(extent_geojson)], crop=True)
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff",
                          "height": out_image.shape[1],
                          "width": out_image.shape[2],
                          "transform": out_transform})
            # only needed for the lat lon coordinates
            with rio.open(tif_path, "w", **out_meta) as dest:
                dest.write(out_image)
            return out_image[0]

In [16]:
# Task 1
def landsat_authentication(env, context):
    return {'apikey' : Phenology.get_apikey(os.environ['USERNAME'], os.environ['PASSWORD'])}

env['predecessor_outputs']['1'] = landsat_authentication(env, context)

In [17]:
# Task 2
def landsat_get_scenes(env, context):
    res = Phenology.get_scenes(env['predecessor_outputs']['1']['apikey'], 
                        os.environ['REGION'], env['static_input']['horizontal'], 
                        env['static_input']['vertical'],
                        env['static_input']['start_date'], 
                        env['static_input']['end_date'])
    scenes = []
    for x in res['data']['results']:
        scenes.append({'date': x['acquisitionDate'], 'entityId': x['entityId']})
    with open(os.environ['GEOJSON']) as f:
        geojson = json.load(f)
    extent_geojson = geojson['features'][0]['geometry']
    return {'response': scenes, 'count': len(scenes), 'extent_geojson': json.dumps(extent_geojson), 'apikey': env['predecessor_outputs']['1']['apikey']}
                                    
env['predecessor_outputs']['2'] = landsat_get_scenes(env, context)

In [18]:
# Task 3
def landsat_get_stats(env, context):
    entityId = env['predecessor_outputs']['2']['response']['entityId']
    date = env['predecessor_outputs']['2']['response']['date']
    tarname, url = Phenology.get_url(entityId, env['predecessor_outputs']['2']['apikey'])
    Phenology.download(tarname, url)
    stats = Phenology.get_statistical_values(tarname)
    return {date : stats}

In [19]:
# this should be parallelized
stats = []
res = env['predecessor_outputs']['2']
for x in res['response']:
    env['predecessor_outputs']['2']['response'] = x
    stat = landsat_get_stats(env, context)
    stats.append(stat)
    # I just don't want this to take that long
    if len(stats) == 1:
        break

In [20]:
# Task 4
def landsat_determine_flowering(env, context):
    stats = env['predecessor_outputs']['3']
    results = {}
    for date in stats:
        results[date] = {'Flowering': True 
                        if not isinstance(stats[date]['NDVI']['avg'], str) and stats[date]['NDVI']['avg'] <= .425 and stats[date]['NDVI']['avg'] >= .375 
                        else False, 
                        'Statistics': stats[date]}
    return results

In [21]:
# This should also be parallelized
results = []
for stat in stats:
    env['predecessor_outputs']['3'] = stat
    results.append(landsat_determine_flowering(env, context))

In [22]:
print(results)

[{'2000-11-12': {'Flowering': False, 'Statistics': {'blue_ref': {'avg': 0.39, 'max': 0.76, 'min': 0.25, 'std': 0.12}, 'green_ref': {'avg': 0.4, 'max': 0.79, 'min': 0.24, 'std': 0.13}, 'red_ref': {'avg': 0.48, 'max': 0.85, 'min': 0.32, 'std': 0.12}, 'NDVI': {'avg': -0.86, 'max': -0.83, 'min': -0.9, 'std': 0.02, 'FF': 0.0, 'VF': 1.0, 'coordinates': '[[-121.7248, 46.7723]]'}}}}]
